In [8]:
from astropy.io import fits
from astropy import units as u
import numpy as np
from matplotlib import pyplot as plt
from astropy.visualization import quantity_support
from astropy.wcs import WCS
# specutils packages
from specutils import Spectrum1D
from specutils.analysis import line_flux
from specutils.fitting import fit_generic_continuum
from specutils import SpectralRegion
from specutils.analysis import equivalent_width
from specutils.analysis import centroid
from specutils.analysis import moment
from specutils.manipulation import extract_region
import warnings
with warnings.catch_warnings():  # Ignore warnings
    warnings.simplefilter('ignore')
quantity_support()

<astropy.visualization.units.quantity_support.<locals>.MplQuantityConverter at 0x7f5843828dc0>

In [9]:
 
hdu = fits.open("../Spectra-lamostdr7/spec-56581-VB031N50V1_sp08-218.fits")
hdudata = hdu[0].data
wl = hdudata[2]
Flux = hdudata[0]

In [10]:
# Defining units astropy
rel_flux = u.def_unit('Relative~flux')
rel_flux.decompose()

Unit("Relative~flux")

In [11]:
lamb = wl * u.AA 
flux = Flux * rel_flux
spec = Spectrum1D(spectral_axis=lamb, flux=flux) 

In [12]:
#Spliting the spectrum en blue and red part
sub_region_blue = SpectralRegion(spec.spectral_axis.min(),  5800*u.AA)
sub_region_red = SpectralRegion(5800*u.AA, spec.spectral_axis.max())
sub_spectrum_blue = extract_region(spec, sub_region_blue)
sub_spectrum_red = extract_region(spec, sub_region_red)

In [14]:
# Subtracting the continuum
with warnings.catch_warnings():  # Ignore warnings
    warnings.simplefilter('ignore')
    g1_fit_blue = fit_generic_continuum(sub_spectrum_blue)
    g1_fit_red = fit_generic_continuum(sub_spectrum_red)
y_continuum_fitted_blue = g1_fit_blue(sub_spectrum_blue.spectral_axis)
spec_sub_blue = sub_spectrum_blue - y_continuum_fitted_blue
y_continuum_fitted_red = g1_fit_red(sub_spectrum_red.spectral_axis)
spec_sub_red = sub_spectrum_red - y_continuum_fitted_red

spec_subtrated =  np.concatenate([spec_sub_blue.flux.value, spec_sub_red.flux.value])
flux_sub = spec_subtrated * rel_flux
spec_sub = Spectrum1D(spectral_axis=lamb, flux=flux_sub)

In [15]:
line_flux(spec_sub, regions=SpectralRegion(6556 * u.AA,  6569* u.AA))  

<Quantity 19336.67464182 Angstrom Relative~flux>

In [20]:
line_flux(spec_sub, regions=SpectralRegion((5411.094 - 3) * u.AA,  (5411.094 + 3)* u.AA))  

<Quantity 623.76827131 Angstrom Relative~flux>